# 🧹 Campaign Data Preprocessing Pipeline
**Dataset:** `Marketing_campaign_dataset.csv` (Rahul Chavan)

## Objetivo
Este notebook limpa e prepara o **Dataset Financeiro de Campanhas**. Este dataset fornece as restrições orçamentais e os custos para o nosso algoritmo de otimização (PLNTDIA).

## Passos Principais
1.  **Carregar Dados:** Importar o CSV original.
2.  **Limpeza de Colunas:** Remover IDs e metadados irrelevantes.
3.  **Tratamento de Valores em Falta:**
    * **Dados Financeiros:** Remover linhas se o Budget estiver em falta (Ground Truth).
    * **Metadados:** Imputar Mediana/Moda.
4.  **Normalização de Texto:** Extrair a `category_key` das hashtags para corresponder ao User Data.
5.  **Features Temporais:** Extrair Dia e Fim de Semana.

In [10]:
import pandas as pd
import numpy as np

marketing_data = pd.read_csv("../Datasets/Marketing_campaign_dataset.csv")
print(f"Original Shape: {marketing_data.shape}")
print("Columns:", marketing_data.columns.tolist())

marketing_data['time'] = pd.to_datetime(marketing_data['time'])

#Since there is only one value to these columns they will be dropped
exchange_rate_unique = marketing_data["exchange_rate"].unique()
print("Unique values in exchange_rate:", exchange_rate_unique)
network_margin_unique = marketing_data["network_margin"].unique()
print("Unique values in network_margin:", network_margin_unique)
max_bid_unique = marketing_data["max_bid_cpm"].unique()
print("Unique values in max_bid_cpm:", max_bid_unique)

#Since these columns are empty they will be dropped
position_in_content_unique = marketing_data["position_in_content"].unique()
unique_reach_unique = marketing_data["unique_reach"].unique()
total_reach_unique = marketing_data["total_reach"].unique()

print("Unique values in position_in_content:", position_in_content_unique)
print("Unique values in unique_reach:", unique_reach_unique)
print("Unique values in total_reach:", total_reach_unique)

marketing_data.drop(columns=["ext_service_id", "creative_id", "creative_width", "creative_height", "search_tags",
                             "template_id", "landing_page", "advertiser_id", "advertiser_name", 
                             "network_id", "channel_id", "advertiser_currency", "channel_id", "max_bid_cpm", 
                             "network_margin", "stats_currency", "currency_code", "exchange_rate", "cmi_currency_code",
                             "position_in_content", "unique_reach", "total_reach", "keywords"], inplace=True)

# Drop row duplicates (if any)
marketing_data = marketing_data.drop_duplicates()

print(f"New Shape: {marketing_data.shape}")
print("Columns:", marketing_data.columns.tolist())
marketing_data.head()
marketing_data.info()

Original Shape: (72612, 35)
Columns: ['campaign_item_id', 'no_of_days', 'time', 'ext_service_id', 'ext_service_name', 'creative_id', 'creative_width', 'creative_height', 'search_tags', 'template_id', 'landing_page', 'advertiser_id', 'advertiser_name', 'network_id', 'approved_budget', 'advertiser_currency', 'channel_id', 'channel_name', 'max_bid_cpm', 'network_margin', 'campaign_budget_usd', 'impressions', 'clicks', 'stats_currency', 'currency_code', 'exchange_rate', 'media_cost_usd', 'position_in_content', 'unique_reach', 'total_reach', 'search_tag_cat', 'cmi_currency_code', 'timezone', 'weekday_cat', 'keywords']
Unique values in exchange_rate: [1]
Unique values in network_margin: [0.]
Unique values in max_bid_cpm: [nan  1.]
Unique values in position_in_content: [nan]
Unique values in unique_reach: [nan]
Unique values in total_reach: [nan]
New Shape: (72612, 13)
Columns: ['campaign_item_id', 'no_of_days', 'time', 'ext_service_name', 'approved_budget', 'channel_name', 'campaign_budget_u

In [11]:
numeric_cols = marketing_data.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = marketing_data.select_dtypes(include=['object']).columns

for col in marketing_data.columns:
    print(f"number of missing values in {col}: {marketing_data[col].isnull().sum()}")

    # We want to handle Ordinal features specifically later.
    # So we skip them in this general loop.
    if col in ['time', 'keywords']:
        continue
    
    # Check if column has missing values
    if marketing_data[col].isnull().sum() > 0:
        
        # --- STRATEGY A: DROP (For Target Variables) ---
        if col in ['campaign_budget_usd', 'approved_budget']:
            # WHY: These variables define the 'Fitness Function' (Profit/Loss).
            # If we impute (guess) these values, we might accidentally label a non-converting user as profitable (e.g., filling with average ROI).
            print(f"Dropping rows with missing {col} (Critical Ground Truth)...")
            marketing_data = marketing_data.dropna(subset=[col])
        
        # --- STRATEGY B: IMPUTE MEDIAN (For Numerical Features) ---
        elif col in numeric_cols:
            # WHY MEDIAN: Marketing data often has outliers (e.g., 'view_time' where a user leaves a tab open for hours). 
            # The Mean is sensitive to these outliers (pulling the average up to unrealistic levels).
            # The Median represents the 'typical' user behavior more accurately.
            median_val = marketing_data[col].median()
            marketing_data[col] = marketing_data[col].fillna(median_val)
            
        # --- STRATEGY C: IMPUTE MODE (For Categorical Features) ---
        elif col in categorical_cols:
            # WHY MODE: For categories like 'Location' or 'Device', we cannot calculate an average. 
            # The safest statistical assumption is the most Frequent Value (Mode), as it is the most probable category for an unknown user.
            mode_val = marketing_data[col].mode()[0]
            marketing_data[col] = marketing_data[col].fillna(mode_val)

 # WHY WE DON'T DROP ROWS:
    # 1. Statistical Power: Dropping rows reduces sample size. If multiple columns have small gaps, we could lose 30-50% of the data.
    # 2. Bias Prevention: If data is missing systematically (e.g., mobile  devices failing to log 'view_time'), dropping rows would bias the 
    #    model against that group (Mobile users).

print("Missing values handled.")

number of missing values in campaign_item_id: 0
number of missing values in no_of_days: 0
number of missing values in time: 0
number of missing values in ext_service_name: 0
number of missing values in approved_budget: 406
Dropping rows with missing approved_budget (Critical Ground Truth)...
number of missing values in channel_name: 0
number of missing values in campaign_budget_usd: 0
number of missing values in impressions: 0
number of missing values in clicks: 0
number of missing values in media_cost_usd: 0
number of missing values in search_tag_cat: 0
number of missing values in timezone: 0
number of missing values in weekday_cat: 0
Missing values handled.


In [12]:
# Extract Day of Week (0=Monday, 6=Sunday)
# WHY: Weekend behavior (gaming, shopping) differs from workday behavior (news, business).
marketing_data['day_of_week'] = marketing_data['time'].dt.dayofweek

# Create a 'Is_Weekend' Binary Feature
# WHY: This simplifies the signal for the model. 
marketing_data['is_weekend'] = marketing_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

# Drop original time-related columns that are no longer needed since we dont have an hour we cant use timezone and the weekday_cat can be seperated into day_of_week and is_weekend
marketing_data.drop(columns=["timezone", "weekday_cat"], inplace=True)

print("Time features extracted.")
marketing_data[['time', 'day_of_week', 'is_weekend']].head()
marketing_data.info()

Time features extracted.
<class 'pandas.core.frame.DataFrame'>
Index: 72206 entries, 0 to 72611
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   campaign_item_id     72206 non-null  int64         
 1   no_of_days           72206 non-null  int64         
 2   time                 72206 non-null  datetime64[ns]
 3   ext_service_name     72206 non-null  object        
 4   approved_budget      72206 non-null  float64       
 5   channel_name         72206 non-null  object        
 6   campaign_budget_usd  72206 non-null  float64       
 7   impressions          72206 non-null  int64         
 8   clicks               72206 non-null  int64         
 9   media_cost_usd       72206 non-null  float64       
 10  search_tag_cat       72206 non-null  object        
 11  day_of_week          72206 non-null  int32         
 12  is_weekend           72206 non-null  int64         
dtypes: datetime

In [13]:
ext_service_name = marketing_data["ext_service_name"].unique()
channel_name = marketing_data["channel_name"].unique()
search_tag_cat = marketing_data["search_tag_cat"].unique()

print("Unique Device Types:", ext_service_name)
print("Unique Channel Names:", channel_name)
print("Unique Search Tag Categories:", search_tag_cat)

Unique Device Types: ['Facebook Ads' 'DV360' 'Google Ads']
Unique Channel Names: ['Mobile' 'Social' 'Video' 'Display' 'Search']
Unique Search Tag Categories: ['Others' 'Youtube' 'Inmarket' 'Retargeting' 'Affinity']


In [14]:
categorical_cols_to_clean = ['ext_service_name', 'channel_name', 'search_tag_cat']

# Definir um limite (threshold).
# Categorias que aparecem em menos de 1% (0.01) das linhas serão transformadas em "Other".
threshold_percent = 0.01

for col in categorical_cols_to_clean:
    # Preencher valores vazios (NaN) com "Other"
    # Isto resolve o problema de dados futuros virem vazios
    marketing_data[col] = marketing_data[col].fillna('Other')
    
    # Identificar categorias raras
    # value_counts(normalize=True) devolve a percentagem de cada categoria
    counts = marketing_data[col].value_counts(normalize=True)
    
    # Criar uma lista com os nomes das categorias que estão abaixo do limite
    rare_categories = counts[counts < threshold_percent].index
    
    # Substituir as categorias raras por "Other"
    if len(rare_categories) > 0:
        print(f"Coluna '{col}': Agrupando {len(rare_categories)} categorias raras em 'Other'.")
        marketing_data.loc[marketing_data[col].isin(rare_categories), col] = 'Other'
    else:
        print(f"Coluna '{col}': Nenhuma categoria rara encontrada abaixo de {threshold_percent*100}%.")

# Verificar como ficaram os valores únicos agora
print("-" * 30)
for col in categorical_cols_to_clean:
    print(f"Unique values in {col} (Cleaned): {marketing_data[col].unique()}")


Coluna 'ext_service_name': Nenhuma categoria rara encontrada abaixo de 1.0%.
Coluna 'channel_name': Nenhuma categoria rara encontrada abaixo de 1.0%.
Coluna 'search_tag_cat': Agrupando 1 categorias raras em 'Other'.
------------------------------
Unique values in ext_service_name (Cleaned): ['Facebook Ads' 'DV360' 'Google Ads']
Unique values in channel_name (Cleaned): ['Mobile' 'Social' 'Video' 'Display' 'Search']
Unique values in search_tag_cat (Cleaned): ['Others' 'Youtube' 'Inmarket' 'Retargeting' 'Other']


In [15]:
# ONE-HOT ENCODING (For Nominal Categories)
# WHY: For 'ext_service_name', 'channel_name', 'search_tag_cat', there is no order.
# We convert these into binary columns (is_DV360).
# This prevents the model from assuming false relationships.

# Identify columns to encode
categorical_cols = ['ext_service_name', 'channel_name', 'search_tag_cat']

# Apply One-Hot Encoding
# drop_first=True reduces redundancy (e.g., if is_Male=0, we know it's Female).
marketing_data = pd.get_dummies(marketing_data, columns=categorical_cols, drop_first=True)
# Convert boolean (True/False) to Integer (1/0) for consistency
marketing_data = marketing_data.replace({True: 1, False: 0})

print(f"Encoding Complete. Final Column Count: {len(marketing_data.columns)}")
marketing_data.head()

Encoding Complete. Final Column Count: 20


C:\Users\gonal\AppData\Local\Temp\ipykernel_38148\4042332237.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  marketing_data = marketing_data.replace({True: 1, False: 0})


,campaign_item_id,no_of_days,time,approved_budget,campaign_budget_usd,impressions,clicks,media_cost_usd,day_of_week,is_weekend,ext_service_name_Facebook Ads,ext_service_name_Google Ads,channel_name_Mobile,channel_name_Search,channel_name_Social,channel_name_Video,search_tag_cat_Other,search_tag_cat_Others,search_tag_cat_Retargeting,search_tag_cat_Youtube
0,2733,7,2022-05-01,400.0,652.173913,837,8,14.058514,6,1,1,0,1,0,0,0,0,1,0,0
1,2733,8,2022-05-02,400.0,652.173913,2634,44,99.633496,0,0,0,0,0,0,1,0,0,1,0,0
2,2733,9,2022-05-03,400.0,652.173913,2135,32,109.419677,1,0,1,0,0,0,1,0,0,1,0,0
3,2733,10,2022-05-04,400.0,652.173913,2327,48,115.209499,2,0,1,0,0,0,0,1,0,1,0,0
4,2733,11,2022-05-05,400.0,652.173913,1538,20,66.990104,3,0,0,1,1,0,0,0,0,1,0,0


In [16]:
# --- FINAL SANITY CHECK ---
# Ensure all data is numeric (except timestamp) and no NaNs exist.
print(f"Final Shape: {marketing_data.shape}")
print(f"Remaining Missing Values: {marketing_data.isnull().sum().sum()}")

# Check data types to ensure everything is int/float
print(marketing_data.dtypes)

Final Shape: (72206, 20)
Remaining Missing Values: 0
campaign_item_id                          int64
no_of_days                                int64
time                             datetime64[ns]
approved_budget                         float64
campaign_budget_usd                     float64
impressions                               int64
clicks                                    int64
media_cost_usd                          float64
day_of_week                               int32
is_weekend                                int64
ext_service_name_Facebook Ads             int64
ext_service_name_Google Ads               int64
channel_name_Mobile                       int64
channel_name_Search                       int64
channel_name_Social                       int64
channel_name_Video                        int64
search_tag_cat_Other                      int64
search_tag_cat_Others                     int64
search_tag_cat_Retargeting                int64
search_tag_cat_Youtube             

In [17]:
# --- SAVE THE DATASET ---
marketing_data.to_csv("../Cleaned_Datasets/Marketing/marketing_cleaned_prepared.csv")
print("Dataset successfully cleaned, encoded, and saved.")

Dataset successfully cleaned, encoded, and saved.
